# Lecture 3: Deutsch-Josza algorithm

<center><img src="../figures/qibo_mascotte/qibo_dj.png" alt="drawing" width="400"/></center>
<center><strong>Fig. 3:</strong> Qibo the mangoose cheking if a target is balanced [DALL-E].</center>


## Introduction

One of the first algorithms in which quantum computing has been proven to be more effective than classical computing is the Deutsch-Josza algorithm. 

The goal of this algorithm is to understand if a target function $f$ is constant or balanced.
<center><img src="../figures/dj.svg" alt="drawing" width="800"/></center>
Since we want to use the quantum computing to tackle the problem, we use to formalize it in the following way.

1. let's consider a function $f$ which takes as input a bitstring $x$ and returns one single value between zero and one: $$ f(x): \{0,1\}^N \to \{0,1\};$$
2. we define it constant if $f(x)=0$ or $f(x)=1$ independently of the bitstring $x$;
3. on the other hand, we define it balanced if it returns half of the time zero and half of the time one.

#### The main ingredients of DJ algorithm

We are going to implement the DJ algorithm considering $N$-long input bitstrings. To do this, we need to initialize an $N+1$ qubits circuit, in which the first $N$ qubits (also called **input register**) are used to encode the bitstring, while the last qubit is used as ancilla. 

We denote the input register state with $|x\rangle$ and the ancilla state with $|y\rangle$.
<center><img src="../figures/dj-circ.png" alt="drawing" width="500"/></center>

At this point, we apply the following steps:
1. we prepare the **input register** in the $|0\rangle^{\otimes N}$ and the ancilla into the $|1\rangle$;
2. we apply an Hadamard gate to each qubit of the system, so that we create a maximally superposed state;
3. we apply an **oracle** operation, which has to be implemented so that it represents our input string. At this stage, we can see the oracle as a black-box whose action on the system is: $$ |x\rangle |y\rangle \to |x\rangle |y \oplus f(x)\rangle,$$ but we are going to open the black-box later in the lecture.
4. we apply again an Hadamard gate to each qubit of the **input register**;
5. we measure the state of the **input register**.
6. if the first $N$ qubits are in the $|0\rangle^{\otimes N}$, the function $f$ is detected as constant, otherwise it is detected as balanced.

#### Some useful imports

In [ ]:
# if you don't have already qibo, qibojit and qiboedu installed in your computer
# uncomment and execute the following lines 

# !pip install qibo
# !pip install qibojit
# !pip install git+https://github.com/qiboteam/qiboedu

In [ ]:
# import qibo and its objects

# import useful plotscripts
# plot_input_register_amplitudes 

# set the backend

#### Start building DJ algorithm

As first step we can build the DJ algorithm parts which are well defined independently from the oracle function.

In [ ]:
# set the number of qubits (for example 3)

In [ ]:
# we will build the whole circuit as sum of: state preparation + oracle + final hadamards and measurements

# initial layer 

# final layer   

We can start building the DJ circuit by adding the initial layer.

In [ ]:
# build the initial layer and draw the circuit

We can have a look to the amplitudes of the first $N$ qubits of the system using a plotscript you can import as `scripts.plotscripts.plot_input_register_amplitudes`. This function computes the amplitude of the first $N$ qubits of an $N+1$ system of qubits removing the dependence on the ancilla.

In [ ]:
# to plot the amplitudes we need to save the qibo state object

# then we can call plot_input_register_amplitudes

As you can see from the plot, the system is in the complete superposed state we get applying the Hadamards to the state $|0\rangle^{\otimes N}$.

Let's have a look to the final layer too:

In [ ]:
# build final layer circuit and draw it

### Opening the black box: oracle definition

We need to define into a sequence of gates a function $f$ which can be balanced or constant acting as $$ |x\rangle |y\rangle \to |x\rangle |y \oplus f(x)\rangle \\.$$ We are going to implement it as by cases.

#### Constant oracle

The easiest way to implement a function $f$ which always returns $0$ indipendently of $|x\rangle$ is to leave the system as it is, without applying any operation on the ancilla. This is equivalent to apply a function which constantly returns zero.

In [ ]:
# define the constant oracle function

In [ ]:
# build constant oracle and draw it

What we expect from a constant oracle of this nature is that the initial state is untouched. This can be verified by plotting the input register's amplitudes.

In [ ]:
# what do we get after initial_layer + constant_oracle?
# build circuit, draw it

In [ ]:
# collect state object and visualize state through plot_input_register_amplitudes

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
    <strong>Exercise:</strong> how to create the oracle corresponding to a constant function f(x)=1?
</div>

#### Balanced oracle

A balanced function can be implemented by applying CNOT gates using a control qubit which is in a complete superposed state. In fact, when we apply an Hadamard gate to a qubit prepared into the zero state, we get the superposed one: $$ |\psi\rangle \frac{1}{\sqrt{2}} (|0\rangle + |1\rangle),$$
and if we use it as control while applying a controlled gate, it will act as a trigger by applying the operator with an exact probability of 50%, corresponding to the state amplitude related to state $|1\rangle$.

In [ ]:
# define the balanced oracle function
# we are going to build a flexible balanced oracle, which can be customized
# by passing as argument a bitstring

In [ ]:
# build balanced oracle and draw it

# you can set a desired string, e.g. "100"

If the balanced oracle is acting in a proper way, it should flip the amplitudes of half states of the system (the ones corresponding to the cases in which an odd number of the control qubits occupy the state $|1\rangle$). We can check it by plotting the amplitudes after applying `initial_layer + balanced_oracle`.

In [ ]:
# what do we get after initial_layer + balanced_oracle?

# build circuit, draw it

In [ ]:
# collect state object and visualize state through plot_input_register_amplitudes

As we can see from the plot, the flipped amplitudes correspond to the ones in which the control qubits are in the state $|1\rangle$

#### Combining all layers into the DJ algorithm

Finally, we can combine all the layers and verify the effectiveness of the DJ algorithm. We expect to get $|0\rangle^{\otimes N}$ if the oracle is constant and, on the other side, probability zero of getting $|0\rangle^{\otimes N}$ if the oracle corresponds to a balanced function.

#### DJ and constant oracle

In [ ]:
# full algorithm with constant oracle: initial layer + constant oracle + final layer

In [ ]:
# draw the circuit

In [ ]:
# collect outcome object

# print state

And what about the final amplitudes of the input register?

In [ ]:
# plot the state via plot_input_register_amplitudes

#### DJ and balanced oracle

In [ ]:
# full algorithm with constant oracle: initial layer + balanced oracle + final layer

In [ ]:
# draw the circuit

In [ ]:
# collect outcome object

# print state

In [ ]:
# plot the state via plot_input_register_amplitudes

#### But wait, what does black box means?

We often speak of the oracles as black boxes: something you are not expected to look into. In this lecture, on contrary, we built our own oracles, perfectly knowing what we have inside. 

This is true! Because we really wanted to understand what a balanced and a constant oracle are. But once this is done, we can finally use the DJ algorithm with is pure power. Thanks to DJ you are able to understand if the function encoded into an oracle is balanced or constant **without** looking into the oracle and, moreover, with just one algorithm evaluation. 

Let's make a simple exercise!

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
<strong>Exercise:</strong> import the <code>qiboedu.scripts.utils.black_box_oracle</code> function. It will return a simple oracle function. It accepts one argument `n`, exactly like the functions you already used during this lecture. Let's build a DJ circuit around this black box oracle and execute it. Can you understand if the encoded function is balanced or constant? Finally, open the black box and check with your eyes.
</div>